In [1]:
from keras.datasets import imdb

In [2]:
(train_data,train_label),(test_data,test_label) = imdb.load_data(num_words=10000)

In [3]:
import numpy as np

def vectorize_sequences(sequences,dimensions=10000):
    results = np.zeros((len(sequences),dimensions))
    for i,sequence in enumerate(sequences):
        results[i,sequence]=1
        
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)


In [4]:
y_train = np.asarray(train_label).astype('float32')
y_test= np.asarray(test_label).astype('float32')

In [5]:
from keras.models import Sequential
from keras.layers import Dense

In [6]:
model = Sequential()

model.add(Dense(16,input_shape=(10000,),activation='relu'))
model.add(Dense(16,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │       160,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,305 (626.19 KB)

 Trainable params: 160,305 (626.19 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
history = model.fit(x_train,y_train,validation_split=0.2,epochs=20,verbose=1,batch_size=512)

Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7101 - loss: 0.2033 - val_accuracy: 0.8844 - val_loss: 0.0978
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9166 - loss: 0.0743 - val_accuracy: 0.8928 - val_loss: 0.0821
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9417 - loss: 0.0500 - val_accuracy: 0.8900 - val_loss: 0.0821
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9637 - loss: 0.0368 - val_accuracy: 0.8886 - val_loss: 0.0834
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9696 - loss: 0.0311 - val_accuracy: 0.8870 - val_loss: 0.0851
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9799 - loss: 0.0231 - val_accuracy: 0.8814 - val_loss: 0.0882
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9837 - loss: 0.0194 - val_accuracy: 0.8810 - val_loss: 0.0921
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9881 - loss: 0.0160 - val_accuracy: 0.8784 - val_loss:

In [10]:
mse,mae = model.evaluate(x_test,y_test)

print('MSE ',mse)
print('MAE ',mae)

782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 310us/step - accuracy: 0.8588 - loss: 0.1175
MSE  0.11874201893806458
MAE  0.858959972858429


In [11]:
y_preds = model.predict(x_test)

782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 331us/step


In [12]:
y_preds

array([[7.7475682e-02],
       [9.9999785e-01],
       [7.8615767e-04],
       ...,
       [3.3708683e-01],
       [3.7875352e-03],
       [2.4081267e-02]], dtype=float32)

In [13]:
y_test

array([0., 1., 1., ..., 0., 0., 0.], dtype=float32)

In [14]:
tests=[]
for i in y_test:
    tests.append(int(i))
preds=[]
for i in y_preds:
    if i[0]>0.5:
        preds.append(1)
    else:
        preds.append(0)
from sklearn.metrics import accuracy_score,precision_score,recall_score

print(accuracy_score(tests,preds))
print(precision_score(tests,preds))
print(recall_score(tests,preds))

0.85896
0.8659869494290375
0.84936


In [15]:
word_index = imdb.get_word_index()
def return_token(tid):
    for k,v in word_index.items():
        # We decode the review; note that our indices were offset by 3
        # because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
        if v==tid-3:
            return k
    return '?'
def print_review(id_):
    sentence = ' '.join(return_token(i) for i in train_data[id_])
    return sentence
print_review(0)

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [16]:
def print_review(id_):
    sentence = ' '.join(return_token(i) for i in train_data[id_])
    return sentence

In [17]:
print_review(0)

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [18]:
train_label[0] #Positive

1

In [19]:
print_review(1)

"? big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal ? the hair is big lots of boobs ? men wear those cut ? shirts that show off their ? sickening that men actually wore them and the music is just ? trash that plays over and over again in almost every scene there is trashy music boobs and ? taking away bodies and the gym still doesn't close for ? all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then"

In [20]:
train_label[1] # Negative

0

In [21]:
print_review(2)

"? this has to be one of the worst films of the 1990s when my friends i were watching this film being the target audience it was aimed at we just sat watched the first half an hour with our jaws touching the floor at how bad it really was the rest of the time everyone else in the theatre just started talking to each other leaving or generally crying into their popcorn that they actually paid money they had ? working to watch this feeble excuse for a film it must have looked like a great idea on paper but on film it looks like no one in the film has a clue what is going on crap acting crap costumes i can't get across how ? this is to watch save yourself an hour a bit of your life"

In [22]:
train_label[2] # Negative

0